# Client Manager Adminstrator
#### DynamoDB Table - tps-&lt;teamName&gt;-CustomerConfig-&lt;env&gt;

## Client grouping concept
### This notebook is used for onboarding clients for each data lake team space
### Each Client should be configured in the same AWS Region as the AMC INSTANCE.
### Each client is defined by 
#### 1. AMC instance
#### 2. Corresponding profiles ids which are grouped for this customer according the advertiserids used for setting up the AMC instance

#### -----------------------

## Global Configs
#### Edit the following configuration for your particular AMC Instance.

#### -----------------------

In [ ]:
## The Team Name configured in the Data Lake platform
# This is the same value passed in the 'data_lake_parameters' team of ddk.json file
TEAM_NAME = "demoteam"

## The Dataset Name configured in the Data Lake platform
# This is the same value passed in the 'data_lake_parameters' dataset of ddk.json file
amc_dataset_name = "amcdataset"

## Region of the AMC instance
aws_region = "us-east-1"

##The Name of the Child Environment Deployed In
ENV="dev"

# The AMC Instance Information Can Be Found on Your AMC UI Page
amc_api_endpoint = "<ENTER AMC API ENDPOINT URL>"

amc_s3_bucket_name = "<ENTER AMC S3 BUCKET NAME>"

amc_data_upload_acct = "<ENTER ACCOUNT ID>"

In [ ]:
import boto3
if len(aws_region) == 0:
    aws_region = str(boto3.Session().region_name)
print("Region : " + aws_region)

## To view the list of configured client details
#### Run the below cell to view the list

#### -----------------------

In [ ]:
from atsclientslibraries.clientmgr import customer
import pandas as pd
## Pandas display configs
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)

df = customer.get_customers_config(TEAM_NAME=TEAM_NAME)
df.head()

#### -----------------------

## To Insert or Update client details
####        Run the below cell to setup and verify the client configuration
```
{
    "customer_id": <Customer ID which will be used to tag resources like bucket prefixes and table names in the platform. It has to unique and can be the name of the amc instance>,
    
    "customer_name":<Customer Name. This will be used to idenify a customer. Keep it less than 5 characters and avoid any special characters>,
    
    "customer_type": <ENDEMIC or NON-ENDEMIC>,
    
    "region":<Region of the AMC instance. This can be requested or retrieved from the ticket raised to the AMC team>,
    
    "amc":{
        "amc_datset_name":<Name of the dataset which is configured in SDLF Data Lake to process the AMC datasets>,
        
        "endpoint_url":<AMC instance endpoint. This can be requested or retrieved from the ticket raised to the AMC team >,
        
        "aws_orange_account_id":<AMC Orange Account. This can be requested or retrieved from the ticket raised to the AMC team>,
                
        "bucket_name":<Name of the bucket for the AMC Dataset landing bucket in your AWS account. This can be requested or retrieved from the ticket raised to the AMC team>
    }
}
```

In [ ]:
import json


customer_details = {
    "customer_id":"democustomer",
    "customer_name":"DemoCustomer",
    "customer_type": "ENDEMIC",
    "region":aws_region,
    "amc":{
        "amc_dataset_name":amc_dataset_name,
        "endpoint_url": amc_api_endpoint,
        "aws_orange_account_id": amc_data_upload_acct,
        "bucket_name": amc_s3_bucket_name
    }
}

print(json.dumps(customer_details, indent=4))

### Run the below cell to submit the client configuration. 
#### Before submitting verify the configuration by running the above cell

Submitting the client configuration will create the client's AMC S3 bucket and configure / onboard the client.

In [ ]:
from atsclientslibraries.clientmgr import customer

import boto3

dynamodb_resp_wr = customer.set_customers_config(customer_details=customer_details, TEAM_NAME=TEAM_NAME, ENV=ENV)
dynamodb_resp_wr